In [1]:
import numpy as np
import pandas as pd

In [2]:
import stock_adv_Functions as advf

In [1]:
from stock_adv_Class import stock_adv as advc

In [2]:
ticket = advc()

In [3]:
ticket.looker('AAPL', 'all', 'relevance', 'month', True)

Stock data downloaded from Yahoo finance
Posts downloaded from Reddit
Comments downloaded from Reddit


None,
  'dividendRate': 0.88,
  'exDividendDate': 1620345600,
  'beta': 1.203743,
  'circulatingSupply': None,
  'startDate': None,
  'regularMarketDayLow': 124.55,
  'priceHint': 2,
  'currency': 'USD',
  'trailingPE': 28.008543,
  'regularMarketVolume': 71311109,
  'lastMarket': None,
  'maxSupply': None,
  'openInterest': None,
  'marketCap': 2079441813504,
  'volumeAllCurrencies': None,
  'strikePrice': None,
  'averageVolume': 98101295,
  'priceToSalesTrailing12Months': 6.3903,
  'dayLow': 124.55,
  'ask': 124.82,
  'ytdReturn': None,
  'askSize': 800,
  'volume': 71311109,
  'fiftyTwoWeekHigh': 145.09,
  'forwardPE': 23.29159,
  'fromCurrency': None,
  'fiveYearAvgDividendYield': 1.37,
  'fiftyTwoWeekLow': 79.3025,
  'bid': 124.61,
  'tradeable': False,
  'dividendYield': 0.0069999998,
  'bidSize': 3100,
  'dayHigh': 125.79,
  'exchange': 'NMS',
  'shortName': 'Apple Inc.',
  'longName': 'Apple Inc.',
  'exchangeTimezoneName': 'America/New_York',
  'exchangeTimezoneShortName': 'E

In [4]:
ticket.sentiment()

2021-05-28 22:16:36,042 loading file C:\Users\Softypo\.flair\models\sentiment-en-mix-distillbert_4.pt


In [8]:
ticket.analyse(period='3d')

In [18]:
posts = ticket.reddit_posts
stock_data = ticket.stock_data
stock_info = ticket.stock_info

In [3]:
stock_info, stock_data, stock_c, stock_r, stock_a, stock_b, stock_qb = advf.y_stock_data('TSLA', '7d', '1h')
posts, comments = advf.pq([stock_info['symbol']+' '+stock_info['longName']], 'all', 'relevance', 'week', True)

In [4]:
advf.sentiment(posts, comments)

2021-05-28 20:25:26,190 loading file C:\Users\Softypo\.flair\models\sentiment-en-mix-distillbert_4.pt


In [50]:
#from stock_adv_Functions import looker

In [51]:
#stock_name, subredits, sort, date, comments = 'TSLA', 'allstocks', 'relevance', 'month', True

#stock_info, stock_data, stock_c, stock_r, stock_a, stock_b, stock_qb, posts, comments = looker(stock_name, subredits, sort, date, comments)

In [22]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#idea based
posts['score'] = posts['score']+1
period = '2d'
sentiment = 'combined_sentiment'
sentiment_prob = 'combined_score'
posts_all_s = posts['score'].loc[posts[sentiment] != np.nan].groupby(posts.loc[posts[sentiment] != np.nan].index.round(period)).sum()

pos_s = posts['score'].loc[posts[sentiment] == 'POSITIVE'].groupby(posts.loc[posts[sentiment] == 'POSITIVE'].index.round(period)).sum()
pos_r = pos_s/posts_all_s
pos_p = posts[sentiment].loc[posts[sentiment] == 'POSITIVE'].groupby(posts.loc[posts[sentiment] == 'POSITIVE'].index.round(period)).count().mean()

neg_s = posts['score'].loc[posts[sentiment] == 'NEGATIVE'].groupby(posts.loc[posts[sentiment] == 'NEGATIVE'].index.round(period)).sum()
neg_r = neg_s/posts_all_s*-1
neg_p = posts[sentiment].loc[posts[sentiment] == 'NEGATIVE'].groupby(posts.loc[posts[sentiment] == 'NEGATIVE'].index.round(period)).count().mean()

date = 'week'

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

#figures
fig.add_trace(go.Candlestick(x=stock_data.index,
                open=stock_data['Open'],
                high=stock_data['High'],
                low=stock_data['Low'],
                close=stock_data['Close'], name = stock_info['symbol']), secondary_y=True)

fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], name = 'Stock price',
                line=dict(color="#ffffff", width=5)),
                secondary_y=True)

fig.add_trace(go.Bar(x=pos_r.index,
                y=pos_r,
                base=0,
                marker_color='lightgreen',
                name='Positive Sentiment',
                text=round(pos_r, 2),
                textposition='auto',
                ), secondary_y=False)

fig.add_trace(go.Bar(x=neg_r.index,
                y=neg_r,
                base=0,
                marker_color='crimson',
                name='Negative Sentiment',
                text=round(neg_r, 2),
                textposition='auto',
                ), secondary_y=False)

fig.update_traces(opacity=0.6)

# Add titles
fig.update_layout(
    title=str(round(stock_data['Close'][-1], 2))+' '+stock_info['currency']+', '+stock_info['longName']+', '+stock_info['symbol']+', '+stock_info['exchange'],
    yaxis_title='Stock Price',
    xaxis_rangeslider_visible=False,
    barmode='relative',
    template='plotly_dark')

# Axes
fig.update_yaxes(showgrid=False)

if date=='week1':
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="day", step="day", stepmode="backward"),
                dict(label="week", step="all"),
                dict(count=7, label="HTD", step="day", stepmode="todate")
            ])
        )
    )

    fig.update()
if date=='month1':
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="week", step="day", stepmode="backward"),
                dict(label="month", step="all"),
                dict(count=1, label="HTD", step="month", stepmode="todate")
            ])
        )
    )

#Show
fig.show()

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#idea based
period = '1d'
sentiment = 'combined_sentiment'
posts_all = posts[sentiment].loc[posts[sentiment] != np.nan].groupby(posts.loc[posts[sentiment] != np.nan].index.round(period)).count()
pos_n = posts[sentiment].loc[posts[sentiment] == 'POSITIVE'].groupby(posts.loc[posts[sentiment] == 'POSITIVE'].index.round(period)).count()/posts_all
neg_n = posts[sentiment].loc[posts[sentiment] == 'NEGATIVE'].groupby(posts.loc[posts[sentiment] == 'NEGATIVE'].index.round(period)).count()/posts_all*-1

date = 'week'

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

#figures
fig.add_trace(go.Candlestick(x=stock_data.index,
                open=stock_data['Open'],
                high=stock_data['High'],
                low=stock_data['Low'],
                close=stock_data['Close'], name = stock_info['symbol']), secondary_y=True)

fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], name = 'Stock price',
                line=dict(color="#ffffff", width=5)),
                secondary_y=True)

fig.add_trace(go.Bar(x=pos_n.index,
                y=pos_n,
                base=0,
                marker_color='lightgreen',
                name='Positive Sentiment',
                text=round(pos_n, 2),
                textposition='auto',
                ), secondary_y=False)

fig.add_trace(go.Bar(x=neg_n.index,
                y=neg_n,
                base=0,
                marker_color='crimson',
                name='Negative Sentiment',
                text=round(neg_n, 2),
                textposition='auto',
                ), secondary_y=False)

fig.update_traces(opacity=0.6)

# Add titles
fig.update_layout(
    title=str(round(stock_data['Close'][-1], 2))+' '+stock_info['currency']+', '+stock_info['longName']+', '+stock_info['symbol']+', '+stock_info['exchange'],
    yaxis_title='Stock Price',
    xaxis_rangeslider_visible=False,
    barmode='relative',
    template='plotly_dark')

# Axes
fig.update_yaxes(showgrid=False)

if date=='week1':
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="day", step="day", stepmode="backward"),
                dict(label="week", step="all"),
                dict(count=7, label="HTD", step="day", stepmode="todate")
            ])
        )
    )

    fig.update()
if date=='month1':
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="week", step="day", stepmode="backward"),
                dict(label="month", step="all"),
                dict(count=1, label="HTD", step="month", stepmode="todate")
            ])
        )
    )

#Show
fig.show()

In [54]:
neg_s

Series([], Name: score, dtype: float64)

In [55]:
pos_s

Series([], Name: score, dtype: float64)

In [63]:
neg_n

Series([], Name: combined_sentiment, dtype: float64)

In [64]:
pos_n

Series([], Name: combined_sentiment, dtype: float64)

In [65]:
posts_all

Series([], Name: combined_sentiment, dtype: int64)

In [66]:
posts['combined_sentiment'].value_counts()

Series([], Name: combined_sentiment, dtype: int64)

In [43]:
posts_g.head()

,title,score,post_id,subreddit,url,num_comments,body,sentiment,comments_sentiment,comments_score,combined_sentiment,combined_score
created,,,,,,,,,,,,
2021-05-24,Why I'm Large and in Charge for Dick Kings (DKNG),0.998371,njaa8g,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,51,"Hey retards, it's been awhile since I've poste...",NEGATIVE,NEGATIVE,0.909460,POSITIVE,0.953916
2021-05-24,"Increased ""competition"" in the EV space is bul...",0.999922,njazsm,stocks,https://www.reddit.com/r/stocks/comments/njazs...,12,**Disclaimer!** I'm not saying i'm bullish (or...,NEGATIVE,NEGATIVE,0.941806,POSITIVE,0.970864
2021-05-18,Michael Burry of ‘The Big Short’ reveals a $53...,0.981055,new7su,stocks,https://www.reddit.com/r/stocks/comments/new7s...,371,\n\n* Michael Burry is long puts against 800...,NEGATIVE,NEGATIVE,0.930124,POSITIVE,0.955590
2021-05-19,Worst buys you’ve made this year so far,0.999982,nfrmgh,stocks,https://www.reddit.com/r/stocks/comments/nfrmg...,643,"I’m new to investing, and like any new investo...",NEGATIVE,NEGATIVE,0.919327,POSITIVE,0.959654
2021-05-17,How am I doing?,0.999076,ndq2ig,stocks,https://www.reddit.com/r/stocks/comments/ndq2i...,154,"I’m 19, going to College for my bachelors in E...",NEGATIVE,NEGATIVE,0.934317,POSITIVE,0.966697


In [10]:
posts.groupby(posts.index.round('D')).mean()

,score,num_comments,comments_score,combined_score
created,,,,
2021-05-17,0.575213,1,NaN,NaN
2021-05-18,0.996458,18,0.936951,0.966704
2021-05-22,0.892782,16,0.919081,0.958218
2021-05-23,0.690700,2,0.938495,0.814597


In [39]:
posts['score'].loc[posts['sentiment'] == 'NEGATIVE'].groupby(posts.loc[posts['sentiment'] == 'NEGATIVE'].index.round('s')).mean()*-1

created
2021-05-18 04:49:31   -0.996458
2021-05-21 17:26:19   -0.997355
2021-05-23 02:57:25   -0.690700
Name: score, dtype: float64

In [80]:
posts['score'].loc[posts['sentiment'] == 'POSITIVE'].groupby(posts.loc[posts['sentiment'] == 'POSITIVE'].index.round('D')).mean()

created
2021-05-17    0.698761
2021-05-18    0.742731
2021-05-19    0.937613
2021-05-20    0.998638
Name: score, dtype: float64

In [58]:
posts['score'].mask(posts['sentiment'] == 'NEGATIVE', posts['score']*-1, inplace=False)

0    -0.981055
1    -0.999982
2    -0.999076
3    -0.994048
4    -0.997254
5     0.794380
6    -0.999984
7     0.575213
8    -0.983755
9    -0.871390
10   -0.841332
11   -0.927603
12   -0.999813
13   -0.834652
14    0.539636
15    0.539636
16   -0.974038
17    0.997866
18    0.981434
19    0.814846
20    0.931593
21   -0.999981
22   -0.999822
23   -0.911607
24   -0.999614
25   -0.999993
26    0.937613
27   -0.999224
28    0.633199
29   -0.997114
30   -0.641618
31    0.999411
32   -0.997114
33   -0.847958
Name: score, dtype: float64

In [59]:
posts['score'].loc[posts['sentiment'] == 'NEGATIVE']*-1

0    -0.981055
1    -0.999982
2    -0.999076
3    -0.994048
4    -0.997254
6    -0.999984
8    -0.983755
9    -0.871390
10   -0.841332
11   -0.927603
12   -0.999813
13   -0.834652
16   -0.974038
21   -0.999981
22   -0.999822
23   -0.911607
24   -0.999614
25   -0.999993
27   -0.999224
29   -0.997114
30   -0.641618
32   -0.997114
33   -0.847958
Name: score, dtype: float64

In [60]:
pd.to_datetime(posts['created'], unit='s')

0    2021-05-18 08:20:11
1    2021-05-19 09:00:45
2    2021-05-16 22:56:29
3    2021-05-22 02:39:05
4    2021-05-21 07:06:44
5    2021-05-18 06:16:24
6    2021-05-21 06:08:35
7    2021-05-16 23:19:23
8    2021-05-18 07:06:49
9    2021-05-21 07:44:06
10   2021-05-18 02:32:27
11   2021-05-18 10:58:56
12   2021-05-21 07:46:46
13   2021-05-20 03:47:54
14   2021-05-17 18:00:16
15   2021-05-17 04:00:17
16   2021-05-20 04:59:36
17   2021-05-20 02:43:28
18   2021-05-17 05:22:38
19   2021-05-18 01:19:16
20   2021-05-18 02:44:20
21   2021-05-18 15:11:54
22   2021-05-22 08:05:20
23   2021-05-21 06:23:44
24   2021-05-22 01:39:22
25   2021-05-21 11:08:21
26   2021-05-18 21:14:22
27   2021-05-18 21:16:46
28   2021-05-18 05:16:07
29   2021-05-17 23:08:06
30   2021-05-18 05:26:55
31   2021-05-20 00:02:12
32   2021-05-17 23:07:33
33   2021-05-21 09:13:05
Name: created, dtype: datetime64[ns]

In [61]:
pd.to_datetime(stock_data.index)

DatetimeIndex(['2021-05-17', '2021-05-18', '2021-05-19', '2021-05-20',
               '2021-05-21'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [20]:
posts, comments = pq(['roblox','RBLX'], 'allstocks', 'relevance', 'week', True)

In [8]:
comments["post_id"].unique()

array(['nerh13', 'ncl6bw'], dtype=object)

In [9]:
posts["post_id"].unique()

array(['nerh13', 'ndqjmt', 'ncl6bw'], dtype=object)

In [10]:
overall_posts = []
overall_posts.append([posts['sentiment'].loc[posts['confidence'] >=0.9].mode()[0], posts['confidence'].loc[posts['confidence'] >=0.9].mean(), posts['title'].loc[(posts['sentiment'] == 'POSITIVE') & (posts['score'] == posts['score'].loc[posts['sentiment'] == 'POSITIVE'].max())], posts['title'].loc[(posts['sentiment'] == 'NEGATIVE') & (posts['score'] == posts['score'].loc[posts['sentiment'] == 'NEGATIVE'].max())]])
overall_posts = pd.DataFrame(overall_posts,columns=['most common sentiment', 'confidence mean', 'top positive post', 'top negative post'])

In [11]:
overall_comments = []
for post_id in comments["post_id"].unique():
    print (post_id)
    print ( comments["sentiment"].loc[(comments["post_id"]==post_id) & (comments["confidence"]>=0.8)].mode()[0] )
    print ( comments["confidence"].loc[(comments["post_id"]==post_id) & (comments["confidence"]>=0.8)].mean() )
    overall_comments.append([post_id, comments["sentiment"].loc[(comments["post_id"]==post_id) & (comments["confidence"]>=0.8)].mode()[0], comments["confidence"].loc[comments["confidence"]>=0.8].mean()])
overall_comments = pd.DataFrame(overall_comments,columns=["post_id", 'most common sentiment', 'confidence mean'])

nerh13
NEGATIVE
0.9833645423253378
ncl6bw
NEGATIVE
0.968096937822259


In [12]:
overall_comments["most common sentiment"].mode()[0]

'NEGATIVE'

In [13]:
posts.loc[posts["confidence"]>=0.8]

,title,score,post_id,subreddit,url,num_comments,body,created,sentiment,confidence,comments_sentiment,comments_confidence,combined_sentiment,combined_confidence
0,"Here is a Market Recap for today Monday, May 1...",67,nerh13,stocks,https://www.reddit.com/r/stocks/comments/nerh1...,18,"**PsychoMarket Recap - Monday, May 17, 2021**\...",1.621313e+09,NEGATIVE,0.996458,NEGATIVE,0.994743,POSITIVE,0.995600
2,$RBLX - DD,31,ncl6bw,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,52,# Introduction\n\nI'm proud to say I've played...,1.621062e+09,NEGATIVE,0.999807,NEGATIVE,0.984415,POSITIVE,0.992111


In [14]:
overall_posts

,most common sentiment,confidence mean,top positive post,top negative post
0,NEGATIVE,0.998132,1 HeWhoSleepsNot random market report 5/16/...,"0 Here is a Market Recap for today Monday, ..."


In [15]:
posts['sentiment'].loc[posts["confidence"]>=0.8].value_counts()

NEGATIVE    2
POSITIVE    0
Name: sentiment, dtype: int64

In [29]:
comments['sentiment'].value_counts()

NEGATIVE    2861
POSITIVE    1421
Name: sentiment, dtype: int64

In [30]:
posts['combined_sentiment'].value_counts()


POSITIVE    21
NEGATIVE    15
Name: combined_sentiment, dtype: int64

In [18]:
posts['title'].loc[posts['confidence'] >=0.9].mode()[0]

'$RBLX - DD'

In [19]:
posts['confidence'].loc[posts['confidence'] >=0.9].mean()

0.9981323480606079

In [20]:
posts['score'].max()

67

In [21]:
posts['title'].loc[(posts['sentiment'] == 'POSITIVE') & (posts['score'] == posts['score'].loc[posts['sentiment'] == 'POSITIVE'].max())]

1    HeWhoSleepsNot random market report 5/16/21
Name: title, dtype: object

In [8]:
comments

,post_id,post,score,comment_id,body,created,sentiment,sentiment_score
0,nlvxzs,Imagine all these trucks with the solar cover!,3,gzn4ihu,Let’s go!!! This Tonneau is awesome.,1.622156e+09,POSITIVE,0.999752
1,nlvxzs,Imagine all these trucks with the solar cover!,2,gznxxei,Yeah this is a set it and forget it for me. I ...,1.622168e+09,NEGATIVE,0.999878
2,nlvxzs,Imagine all these trucks with the solar cover!,2,gzn4q50,Agree...I’m long this one! Best of luck!!,1.622156e+09,POSITIVE,0.710718
3,nlvxzs,Imagine all these trucks with the solar cover!,1,gzoeqod,Totally agree! I love technology we’re not goi...,1.622175e+09,POSITIVE,0.933101
4,nlvxzs,Imagine all these trucks with the solar cover!,3,gzn52i3,"Yep, I think I invested because I want one so ...",1.622156e+09,NEGATIVE,0.976153
...,...,...,...,...,...,...,...,...
66,nkyu3p,SHILL POSTS INCOMING STAY WOKE,2,gzfsfpg,They posted it 4 times AH lol\n\nAnd didn’t me...,1.622006e+09,NEGATIVE,0.813309
67,nkyu3p,SHILL POSTS INCOMING STAY WOKE,2,gzgimlw,Lots of wrinkles in AMC lots of respect for ol...,1.622019e+09,NEGATIVE,0.941478
68,nmnpbj,"Journey to $1 Million - May 28th, 2021",2,gzt153h,I bought some BHP August calls yesterday and t...,1.622266e+09,NEGATIVE,0.999245
69,nmnpbj,"Journey to $1 Million - May 28th, 2021",1,gztmsat,"Nice, I like it.",1.622278e+09,POSITIVE,0.988143


In [7]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     0 non-null      object
 1   post        0 non-null      object
 2   score       0 non-null      object
 3   comment_id  0 non-null      object
 4   body        0 non-null      object
 5   created     0 non-null      object
dtypes: object(6)
memory usage: 80.0+ bytes


In [9]:
comments.empty==True

True

In [9]:
posts["test"] = np.where (posts['comments_sentiment'].isna(), posts['sentiment'], np.where (posts['sentiment'] == posts['comments_sentiment'], 'POSITIVE', 'NEGATIVE'))
#posts["combined_sentiment"] = np.where (posts['sentiment'] == posts['comments_sentiment'], "POSITIVE", "NEGATIVE")

In [7]:
posts

,post_id,title,score,subreddit,url,num_comments,body,sentiment,sentiment_score,comments_sentiment,comments_score,combined_sentiment,combined_score
created,,,,,,,,,,,,,
2021-05-27 08:50:50,nlvxzs,Imagine all these trucks with the solar cover!,9,wksp,https://www.reddit.com/r/wksp/comments/nlvxzs/...,11,"Ford's F-150 electric truck sales hit 70,000: ...",NEGATIVE,0.971231,POSITIVE,0.891190,NEGATIVE,0.931210
2021-05-27 14:17:29,nm138z,Looking for help with xpath,3,sheets,https://www.reddit.com/r/sheets/comments/nm138...,3,"Hi all,\n\nI've been trying to figure out how ...",NEGATIVE,0.990733,POSITIVE,0.974930,NEGATIVE,0.982832
2021-05-27 21:52:03,nm7xjt,"Your Morning Brief for Thursday, May 27, 2021",20,RedditTickers,https://www.reddit.com/r/RedditTickers/comment...,0,#Market Overview\n\nThe S&P 500 is higher Thur...,NEGATIVE,0.998418,NaN,NaN,NEGATIVE,0.998418
2021-05-26 21:47:21,nlhgdw,"Your Morning Brief for Wednesday, May 26, 2021",16,RedditTickers,https://www.reddit.com/r/RedditTickers/comment...,1,#Market Overview\n\nThe S&P 500 is higher Wedn...,NEGATIVE,0.999335,POSITIVE,0.930025,NEGATIVE,0.964680
2021-05-28 21:49:47,nmy95q,"Your Morning Brief for Friday, May 28, 2021",12,RedditTickers,https://www.reddit.com/r/RedditTickers/comment...,1,#Market Overview\n\nThe S&P 500 is higher Frid...,NEGATIVE,0.999724,POSITIVE,0.743806,NEGATIVE,0.871765
2021-05-24 21:51:14,njygdc,"Your Morning Brief for Monday, May 24, 2021",6,RedditTickers,https://www.reddit.com/r/RedditTickers/comment...,1,#Market Overview\n\nThe S&P 500 is higher Mond...,NEGATIVE,0.999651,POSITIVE,0.976070,NEGATIVE,0.987860
2021-05-25 21:47:28,nkq5ns,"Your Morning Brief for Tuesday, May 25, 2021",9,RedditTickers,https://www.reddit.com/r/RedditTickers/comment...,0,#Market Overview\n\nThe S&P 500 is higher Tues...,NEGATIVE,0.998607,NaN,NaN,NEGATIVE,0.998607
2021-05-27 18:32:20,nm4h6p,WKSP on the move and imagine all these EV truc...,2,MillennialBets,https://www.reddit.com/r/MillennialBets/commen...,0,**Author**: u/StAnthonyleadus2it(**Karma:** 44...,NEGATIVE,0.955394,NaN,NaN,NEGATIVE,0.955394
2021-05-27 06:10:33,nlsu85,First real post on WSB - found a walking zombi...,27,wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,32,Redoing this because I tried to emulate instea...,NEGATIVE,0.999991,NEGATIVE,0.949658,POSITIVE,0.974824


In [118]:
for post_id in comments["post_id"].unique():
    posts.at[posts[posts.index==post_id].index,"comments_sentiment"] = comments["sentiment"]["post_id"==post_id].mode()[0]
    posts.at[posts[posts.index==post_id].index,"comments_score"] =comments["sentiment_score"].loc[comments["post_id"]==post_id].mean()
    #posts.at[posts.loc[posts["post_id"]==post_id].index,"comments_sentiment"] = comments["sentiment"].loc[comments["post_id"]==post_id].mode()[0]
    #posts.at[posts.loc[posts["post_id"]==post_id].index,"comments_score"] = comments["sentiment_score"].loc[comments["post_id"]==post_id].mean()

KeyError: False

In [111]:
for post_id in posts[posts["sentiment"].isna()].index:
    #print (posts["sentiment"][['post_id']==post_id] )
    #print (posts[posts['post_id']==post_id].index)
    posts.at[post_id,"sentiment"] = 2

KeyError: 'sentiment'

In [6]:
stock_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2021-04-28 09:30:00-04:00,26.370001,27.094999,26.160000,27.070000,1755500,0,0
2021-04-28 10:30:00-04:00,27.070000,27.350000,27.030001,27.275000,1603922,0,0
2021-04-28 11:30:00-04:00,27.260000,27.389999,27.219999,27.260000,919413,0,0
2021-04-28 12:30:00-04:00,27.264099,27.440001,27.250000,27.320000,769316,0,0
2021-04-28 13:30:00-04:00,27.320000,27.340000,27.160000,27.334999,877649,0,0
...,...,...,...,...,...,...,...
2021-05-27 11:30:00-04:00,31.469999,31.584999,31.410000,31.444401,852549,0,0
2021-05-27 12:30:00-04:00,31.445000,31.570000,31.400000,31.530001,640538,0,0
2021-05-27 13:30:00-04:00,31.525000,31.540001,31.370001,31.430000,761470,0,0
2021-05-27 14:30:00-04:00,31.430000,31.520000,31.405001,31.450001,1024683,0,0


In [75]:
stock_info

{'zip': '77056',
 'sector': 'Energy',
 'fullTimeEmployees': 86000,
 'longBusinessSummary': 'Schlumberger Limited supplies technology for reservoir characterization, drilling, production, and processing to the oil and gas industry worldwide. It operates in four divisions: Digital & Integration, Reservoir Performance, Well Construction, and Production Systems. The company offers reservoir interpretation and data processing services; open and cased-hole, and slickline services; exploration and production pressure and flow-rate measurement services; tubing-conveyed perforating services; integrated production systems; software, consulting, information management, and IT infrastructure services; reservoir characterization, field development planning, and production enhancement consulting services; and petro technical data services and training solutions. It designs, manufactures, and markets roller cone and fixed cutter drill bits; supplies drilling fluid systems, fluid systems and specialty